## Loading models trained for each year

In [3]:
from sklearn.externals import joblib

# year list 
year_list = [2011, 2012, 2013, 2014, 2015, 2016]

# setting up an empty dictionary to deal with feature importance 
feature_imp_dict = {}

for year in year_list:
    file_name = str(year) + "rf.pkl"
    # Loading model file
    mod_file = joblib.load(file_name)
    
    # adding each year feature importance to the dictionary
    if year not in feature_imp_dict:
        feature_imp_dict[year] = sorted(zip(mod_file.feature_importances_, X)) # X here is the list of input features

### Visualization Code

In [19]:
import pandas as pd

# plotly imports
import plotly.graph_objects as go

# Converting the dictionary to a data frame
emp_df = pd. DataFrame()
for year in year_list:
    df = pd.DataFrame(feature_imp_dict[year], columns = ['feature_importance', 'feature'])
    df['year'] = year
    emp_df = pd.concat([emp_df, df])

dataset = emp_df.copy()

# defining the figure dictionary
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# setting the range and title of x axis and y axis
fig_dict["layout"]["xaxis"] = {"range": [0.001, 0.5], "title": "Variable Importance"}
fig_dict["layout"]["yaxis"] = {"title": "Variables"}
fig_dict["layout"]["hovermode"] = "closest"


# defining the buttons on the visualization
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

# defining the year slider
sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}



# defining the initial frame of the visualization
year = 2011

# selecting only top 20 features (This can be parameterized)
dataset_by_year = dataset[dataset["year"] == year].tail(20)

data_dict = go.Bar(
            x=list(dataset_by_year['feature_importance']),
            y=list(dataset_by_year['feature']),
            orientation='h')

fig_dict["data"].append(data_dict)



# # Making different frames of the visualization
for year in year_list:
    frame = {"data": [], "name": str(year)}
    dataset_by_year = dataset[dataset["year"] == year].tail(20)
    data_dict = go.Bar(
            x=list(dataset_by_year['feature_importance']),
            y=list(dataset_by_year['feature']),
            text=list(dataset_by_year['feature']),
            textposition='auto',
            orientation='h')
    frame["data"].append(data_dict)
    

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [year],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": year,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)

fig_dict["layout"]["sliders"] = [sliders_dict]
fig = go.Figure(fig_dict)

# Hiding y_axes lables as they are not dynamic
fig.update_yaxes(visible=False, showticklabels=False)
fig.show()